In [1]:
using JuMP, JuMPeR, Gurobi, LinearAlgebra

## Problema de optimizacion ejemplo ##

$$
\begin{align}
& \text{min} && 8x_1 + 12x_2 \\
& \text{sujeto a} &&  \tilde{a}_{11}x_1 + \tilde{a}_{12}x_2 \geq  140 & &\\
&                   && \tilde{a}_{21}x_1 + \tilde{a}_{22}x_2 \geq  72 && \\
&                   && x_1 \geq 0 &&  \\
&                   && x_2 \geq 0 && 
\end{align}
$$


In [2]:
A = [10 20;6 8]
b = [140;72]
c = [8;12]

mr = RobustModel(solver=GurobiSolver())

@variable(mr,      x[i=1:2] >=0)
@objective(mr, Min, sum(c[i]*x[i] for i=1:2))
@constraint(mr, A*x .>= b)
solve(mr)

getvalue(x)

Academic license - for non-commercial use only
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [6e+00, 2e+01]
  Objective range  [8e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve time: 0.04s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.550000e+01   0.000000e+00      0s
       2    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.05 seconds
Optimal objective  1.000000000e+02


2-element Array{Float64,1}:
 8.0               
 3.0000000000000004

In [2]:
A = [10 20;6 8]
b = [140;72]
c = [8;12]

Au1 = [11 22]
Al1 = [9 18] # peor caso
#Au2 = [6 8]
#Al2 = [6 8]

mr = RobustModel(solver=GurobiSolver())

@variable(mr,      x[i=1:2] >=0)

@uncertain(mr, Al1[i] <= A1r[i=1:2] <= Au1[i])
#@uncertain(mr, Al2[i] <= A2r[i=1:2] <= Au2[i])

@objective(mr, Min, sum(c[i]*x[i] for i=1:2))
@constraint(mr, dot(A1r,x) >= b[1])
#@constraint(mr, dot(A2r,x) >= b[2])
@constraint(mr, dot(A[2,:],x) >= b[2])

solve(mr)

println(getvalue(x))

print(mr)

Academic license - for non-commercial use only
Optimize a model with 4 rows, 6 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve removed 2 rows and 4 columns
Presolve time: 0.06s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.550000e+01   0.000000e+00      0s
       2    1.0311111e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.06 seconds
Optimal objective  1.031111111e+02
[4.88889, 5.33333]
Min 8 x[1] + 12 x[2]
Subject to
 6 x[1] + 8 x[2] >= 72
 9 _π_1_1 + 11 _π_1_2 + 18 _π_1_3 + 22 _π_1_4 <= -140
 _π_1_1 + _π_1_2 + x[1] == 0
 _π_1_3 + _π_1_4 + x[2] == 0
 x[i] >= 0 for all i in {1,2}
 _π_1_1 <= 0
 _π_1_2 >= 0
 _π_1_3 <= 0
 _π_1_4 >= 0
Uncertain constraints:
A1r[1] x[1] + A1r[2] x[2] >= 140
Uncertain parameters:
.. <= A1r[i] <= .. for

In [5]:
A = [10 20;6 8]
b = [140;72]
c = [8;12]
Γ = 2

Au1 = [11 22]
Al1 = [9 18] # peor caso
#Au2 = [6 8]
#Al2 = [6 8]

const UNCERTAINY_SET = :Polyhedral
#const UNCERTAINY_SET = :Ellipsoidal

mr = RobustModel(solver=GurobiSolver())

@variable(mr,      x[i=1:2] >=0)

@uncertain(mr, Al1[i] <= A1r[i=1:2] <= Au1[i])
#@uncertain(mr, Al2[i] <= A2r[i=1:2] <= Au2[i])

@uncertain(mr, A1r[1:2])
# The uncertainty set requires the "square root" of the covariance
μ = A[1,:]  # Want a column vector
L = Diagonal(Array([-1;-2]))  # Σ^½
# Define auxiliary uncertain parameters to model the underlying factors
@uncertain(mr, -1 <= z[1:2] <= 1)
# Link r with z
@constraint(mr, A1r .== L*z + μ)
if UNCERTAINY_SET == :Polyhedral
    @constraint(mr, norm(z,   1) <= Γ)  # ‖z‖₁ ≤ Γ
    @constraint(mr, norm(z, Inf) <= 1)  # ‖z‖∞ ≤ 1
else
    @constraint(mr, norm(z,   2) <= Γ)  # ‖z‖₂ ≤ Γ
end


@objective(mr, Min, sum(c[i]*x[i] for i=1:2))
@constraint(mr, dot(A1r,x) >= b[1])
#@constraint(mr, dot(A2r,x) >= b[2])
@constraint(mr, dot(A[2,:],x) >= b[2])

solve(mr)

println(getvalue(x))

print(mr)

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 10 rows, 19 columns and 43 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 2e+01]
  Objective range  [8e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve removed 4 rows and 10 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 23 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.550000e+01   0.000000e+00      0s
       4    1.0311111e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.00 seconds
Optimal objective  1.031111111e+02
[4.88889, 5.33333]
Min 8 x[1] + 12 x[2]
Subject to
 6 x[1] + 8 x[2] >= 72
 10 _π_1_1 + 20 _π_1_2 + 2 _α′_1_5 + _ω_1_6 + _ω_1_7 - _ω′_1_8 - _ω′_1_9 + 9 _π_1_10 + 11 _π_1_11 + 18 _π_1_12 + 22 _π_1_13 - _π_1_14 + _π_1_15 - _π_1_16 + _π_1_17 <= -140
 _π_1_10 + _π_1_11 == 0
 _π_1_12 + _π_1_

In [26]:
n, C, Γ = 10, 3, 4
p, w = rand(n), rand(n)
σ = 0.2 * rand(n) .* w
knap = RobustModel(solver=GurobiSolver())
@variable(knap,      x[1:n], Bin)
@uncertain(knap, 0 <= z[1:n] <= 1)
@objective(knap, Max, sum{p[i]*x[i], i=1:n})
@constraint(knap, sum{(w[i]+σ[i]*z[i])*x[i], i=1:n} <= C)
@constraint(knap, sum{z[i], i=1:n} <= Γ)
solve(knap)

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 11 rows, 31 columns and 61 nonzeros
Variable types: 21 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [9e-04, 4e+00]
  Objective range  [1e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 11 rows, 31 columns, 61 nonzeros
Variable types: 21 continuous, 10 integer (10 binary)

Root relaxation: objective 3.761315e+00, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.76132    0    1   -0.00000    3.76132      -     -    0s
H    0     0                       3.2499997    3.76132  15.7%     -    0s
H    0     0                       3.5345120    3.76132  6.42%    

:Optimal

In [41]:
#-----------------------------------------------------------------------
# JuMPeR  --  JuMP Extension for Robust Optimization
# http://github.com/IainNZ/JuMPeR.jl
#-----------------------------------------------------------------------
# Copyright (c) 2016: Iain Dunning
# This Source Code Form is subject to the terms of the Mozilla Public
# License, v. 2.0. If a copy of the MPL was not distributed with this
# file, You can obtain one at http://mozilla.org/MPL/2.0/.
#-----------------------------------------------------------------------
# example/cap_budget.jl
# Apply the method from "Multistage Robust Mixed Integer Optimization
# with Adaptive Partitions" by Bertsimas and Dunning to the two-stage
# capital budgeting problem first described in Hanasusanto et. al.'s
# paper "Two-stage robust integer programming".
#
#   max  ∑ᵢ profitᵢ(ξ) [xᵢ + Θ yᵢ(ξ)]
#    st  ∑ᵢ costᵢ(ξ) [xᵢ + yᵢ(ξ)] ≤ B
#        xᵢ, yᵢ(ξ) ∈ {0,1}
#
# Requires a mixed-integer linear optimization problem solver.
#-----------------------------------------------------------------------

using JuMP, JuMPeR, LinearAlgebra, Distributions, Random

"""
    TreeScenario
Stores the values of "active" uncertain parameters, as well as the
associated tree structure described in the paper.
"""
mutable struct TreeScenario
    ξ::Vector{Float64}
    parent
    children::Vector
end
is_leaf(t::TreeScenario) = isempty(t.children)

"""
    generate_data(seed, N)
Generates a set of data for an instance of size `N` using the distributions
described in the paper by Hanasusanto, et. al., where `seed` is the random
seed to use to help with replication.
"""
function generate_data(seed, N)
    Random.seed!(25061985);
    c0 = rand(N)*10  # Nominal costs
    r0 = c0 ./ 5     # Nominal profit
    B  = sum(c0)/2   # Budget
    θ = 0.8          # Profit discount factor for late decisions
    Φ = rand(N,4)    # Factor matrices ...
    Ψ = rand(N,4)    # ... for the uncertainty set
    for i in 1:N
        Φ[i,:] ./= sum(Φ[i,:])
        Ψ[i,:] ./= sum(Ψ[i,:])
    end
    return θ, B, Φ, Ψ, c0, r0
end

"""
    solve_partitioned_problem(N, θ, B, Φ, Ψ, c0, r0, scenarios)
Solve the two-stage problem of size `N` with one partition of the uncertainty
set for every leaf scenario in the `scenario_tree`. At optimality, grows the
tree given the new scenarios obtained.
"""
function solve_partitioned_problem(N::Int, θ::Float64, B::Float64,
                                   Φ::Matrix{Float64}, Ψ::Matrix{Float64},
                                   c0::Vector{Float64}, r0::Vector{Float64},
                                   scenario_tree::Vector{TreeScenario})
    # scenario_tree is a vector containing every scenario in the tree
    # We will have one partition for every leaf scenario in the tree
    leaf_scenarios = filter(is_leaf, scenario_tree)
    P = length(leaf_scenarios)  # Number of partitions
    # Initialize the RO model
    rm = RobustModel(solver=GurobiSolver())
    # Decision variables: whether to pursue a project or not
    # First stage, here-and-now decision
    @variable(rm, x[1:N], Bin)
    # Second stage, wait-and-see decision
    # One set of variables per partition
    @variable(rm, y[1:P,1:N], Bin)
    # Define the uncertain parameters, which belong to a hypercube
    @uncertain(rm, -1 <= ξ[1:4] <= 1)
    # From the uncertain parameters we obtain the cost and profit vectors
    cost   = (1 + 0.5*Φ*ξ) .* c0
    profit = (1 + 0.5*Ψ*ξ) .* r0
    # The objective function will be the minimum of the objective function
    # across all the partitions. Put a default upper bound, just so we don't
    # start off unbounded if we are using a cutting plane method.
    @variable(rm, obj <= 10)
    @objective(rm, Max, obj)
    # For each partition...
    profit_con_refs = []
    budget_con_refs = []
    for p in 1:P
        # Define the partition uncertainty set. We define multiple hyperplanes
        # by walking up the scenario tree from this leaf.
        us = JuMPeR.BasicUncertaintySet()
        current_scenario = leaf_scenarios[p]
        parent_scenario = current_scenario.parent
        # We keep going until we hit the root of the tree, which is a scenario
        # that has no parent
        while parent_scenario != nothing
            for sibling_scenario in parent_scenario.children
                if current_scenario == sibling_scenario
                    continue  # Don't partition against ourself!
                end
                ξ_sub = sibling_scenario.ξ - current_scenario.ξ
                ξ_add = sibling_scenario.ξ + current_scenario.ξ
                @constraint(us, dot(ξ_sub, ξ) <= dot(ξ_sub,ξ_add)/2)
            end
            # Move up the scenario tree
            current_scenario  = parent_scenario
            parent_scenario = current_scenario.parent
        end
        # Constrain objective function for this partition
        profit_con_ref =
            @constraint(rm, obj <= sum{profit[i] * (x[i] + θ*y[p,i]), i in 1:N},
                                uncset=us)
        push!(profit_con_refs, profit_con_ref)
        # Constrain spending for this partition
        budget_con_ref =
            @constraint(rm, sum{cost[i] * (x[i] + y[p,i]), i in 1:N} <= B,
                                uncset=us)
        push!(budget_con_refs, budget_con_ref)
        # Can only choose each project once
        for i in 1:N
            @constraint(rm, x[i] + y[p,i] <= 1)
        end
    end
    # Solve, will use reformulation. We pass disable_cuts=true because
    # we want to be able to use solvers like CBC that don't support lazy
    # constraint callbacks.
    solve(rm, disable_cuts=true, active_scenarios=true)
    # Extend the scenario tree
    for p in 1:P
        # Extract the active uncertain parameter values
        profit_scen = getscenario(profit_con_refs[p])
        profit_scen_ξ = [uncvalue(profit_scen, ξ[i]) for i in 1:4]
        # Create a new child in the tree under this leaf
        profit_child = TreeScenario(profit_scen_ξ, leaf_scenarios[p], [])
        # Same for budget
        budget_scen = getscenario(budget_con_refs[p])
        budget_scen_ξ = [uncvalue(budget_scen, ξ[i]) for i in 1:4]
        budget_child = TreeScenario(budget_scen_ξ, leaf_scenarios[p], [])
        # Add to the tree
        push!(leaf_scenarios[p].children, profit_child)
        push!(leaf_scenarios[p].children, budget_child)
        push!(scenario_tree, profit_child)
        push!(scenario_tree, budget_child)
    end
    # Return the objective function value and the first-stage solution
    getobjectivevalue(rm), getvalue(x)
end

"""
    solve_problem(seed, N)
Solve the problem for the parameters corresponding to the given random seed
and instance size `N`. Prints results are each of five iterations.
"""
function solve_problem(seed, N)
    θ, B, Φ, Ψ, c0, r0 = generate_data(seed, N)
    # Start with no partitions (i.e., one scenario)
    scenario_tree = [ TreeScenario(zeros(4),nothing,[]) ]
    # Iteration 1
    z_iter1, x_iter1 = solve_partitioned_problem(N, θ, B, Φ, Ψ, c0, r0, scenario_tree)
    # Iteration 2
    z_iter2, x_iter2 = solve_partitioned_problem(N, θ, B, Φ, Ψ, c0, r0, scenario_tree)
    # Iteration 3
    z_iter3, x_iter3 = solve_partitioned_problem(N, θ, B, Φ, Ψ, c0, r0, scenario_tree)
    # Iteration 4
    z_iter4, x_iter4 = solve_partitioned_problem(N, θ, B, Φ, Ψ, c0, r0, scenario_tree)
    # Iteration 5
    z_iter5, x_iter5 = solve_partitioned_problem(N, θ, B, Φ, Ψ, c0, r0, scenario_tree)
    @show z_iter1
    println(round.(x_iter1, digits=3))
    @show z_iter2
    println(round.(x_iter2, digits=3))
    @show z_iter3
    println(round.(x_iter3, digits=3))
    @show z_iter4
    println(round.(x_iter4, digits=3))
    @show z_iter5
    println(round.(x_iter5, digits=3))
    @show length(scenario_tree)
end

# By default, solve instance of size 5
solve_problem(106, 5)


Academic license - for non-commercial use only
Optimize a model with 15 rows, 27 columns and 143 nonzeros
Variable types: 17 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [6e-03, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 1.01588 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.015883464221e+00, best bound 1.015883464221e+00, gap 0.0000%
Academic license - for non-commercial use only
Optimize a model with 0 rows, 4 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e-01, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve rem

z_iter1 = 1.015883464220991
[1.0, 1.0, 0.0, 0.0, 0.0]
z_iter2 = 1.2333003055821319
[0.0, 0.0, 0.0, 1.0, 0.0]
z_iter3 = 1.2333003055821319
[0.0, 0.0, 0.0, 1.0, 0.0]
z_iter4 = 1.3764249108594364
[-0.0, -0.0, -0.0, 1.0, -0.0]
z_iter5 = 1.3764249108594364
[0.0, 0.0, -0.0, 1.0, -0.0]
length(scenario_tree) = 63


63

In [6]:
#-----------------------------------------------------------------------
# JuMPeR  --  JuMP Extension for Robust Optimization
# http://github.com/IainNZ/JuMPeR.jl
#-----------------------------------------------------------------------
# Copyright (c) 2016: Iain Dunning
# This Source Code Form is subject to the terms of the Mozilla Public
# License, v. 2.0. If a copy of the MPL was not distributed with this
# file, You can obtain one at http://mozilla.org/MPL/2.0/.
#-----------------------------------------------------------------------
# example/inventory.jl
# Implements the inventory management example from
#   A. Ben-Tal, A. Goryashko, E. Guslitzer, A. Nemirovski
#   "Adjustable Robust Solutions of Uncertain Linear Programs"
# Requires a linear optimization solver.
#-----------------------------------------------------------------------

using JuMP, JuMPeR

# Define problem parameters
I = 3           # Number of factories
T = 24          # Number of time periods
d_nom = 1000*[1 + 0.5*sin(π*(t-1)/12)  for t in 1:T]  # Nominal demand
θ = 0.20        # Uncertainty level
α = [1.0, 1.5, 2.0]  # Production costs
c = [α[i] * (1 + 0.5*sin(π*(t-1)/12))  for i in 1:I, t in 1:T]
P = 567         # Maximimum production per period
Q = 13600       # Maximumum production over all
Vmin = 500      # Minimum inventory at warehouse
Vmax = 2000     # Maximum inventory at warehouse
v1 = Vmin       # Initial inventory (not provided in paper)

# Setup robust model
invmgmt = RobustModel(solver=GurobiSolver())

# Uncertain parameter: demand at each time stage lies in a interval
@uncertain(invmgmt, d_nom[t]*(1-θ) <= d[t=1:T] <= d_nom[t]*(1+θ))

# Decision: how much to produce at each factory at each time
# As this decision can be updated as demand is realized, we will use adaptive
# policy - in particular, an affine policy where production at time t is an
# affine function of the demand realized previously.
@adaptive(invmgmt, p[i=1:I,t=1:T], policy=Affine, depends_on=d[1:t-1])

# Objective: minimize total cost of production
@variable(invmgmt, F)  # Overall cost
@objective(invmgmt, Min, F)
@constraint(invmgmt, F >= sum{c[i,t] * p[i,t], i=1:I, t=1:T})

# Constraint: cannot exceed production limits
for i in 1:I, t in 1:T
    @constraint(invmgmt, p[i,t] >= 0)
    @constraint(invmgmt, p[i,t] <= P)
end
for i in 1:I
    @constraint(invmgmt, sum{p[i,t], t=1:T} <= Q)
end

# Constraint: cannot exceed inventory limits
for t in 1:T
    @constraint(invmgmt,
        v1 + sum{p[i,s], i=1:I, s=1:t} - sum{d[s],s=1:t} >= Vmin)
    @constraint(invmgmt,
        v1 + sum{p[i,s], i=1:I, s=1:t} - sum{d[s],s=1:t} <= Vmax)
end

# Solve
status = solve(invmgmt)

println(getobjectivevalue(invmgmt))

Academic license - for non-commercial use only
Optimize a model with 6100 rows, 12613 columns and 42625 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 3927 rows and 7428 columns
Presolve time: 0.13s
Presolved: 2173 rows, 5185 columns, 30103 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Free vars  : 582
 AA' NZ     : 4.909e+04
 Factor NZ  : 1.305e+05 (roughly 4 MBytes of memory)
 Factor Ops : 1.403e+07 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.62563148e+06 -6.41669392e+04  6.30e+02 1.24e+03  1.19e+05     0s
   1   4.44439632e+06 -7.06291097e+06  5.56e+02 2.70e-04  8.72e+04     0s

Barrier performed 1 itera

In [7]:
print(invmgmt)

Excessive output truncated after 993087 bytes.

In [64]:
#-----------------------------------------------------------------------
# JuMPeR  --  JuMP Extension for Robust Optimization
# http://github.com/IainNZ/JuMPeR.jl
#-----------------------------------------------------------------------
# Copyright (c) 2016: Iain Dunning
# This Source Code Form is subject to the terms of the Mozilla Public
# License, v. 2.0. If a copy of the MPL was not distributed with this
# file, You can obtain one at http://mozilla.org/MPL/2.0/.
#-----------------------------------------------------------------------
# example/portfolio.jl
# Solve a robust portfolio optimization problem. Can choose between a
# polyhedral set (requires a LP solver) or an ellipsoidal set (requires
# a SOCP solver).
#-----------------------------------------------------------------------

using JuMP, JuMPeR  # Modeling
using Distributions  # For generating data
using LinearAlgebra
using Random
using Printf

# Number of stocks
const NUM_ASSET = 10
# Number of samples of past returns
const NUM_SAMPLES = 1000
# Uncertainty set type (:Polyhedral or :Ellipsoidal)
#const UNCERTAINY_SET = :Polyhedral
const UNCERTAINY_SET = :Ellipsoidal
# Seed the RNG for reproducibility
Random.seed!(25061985);

"""
    generate_data()
Sample returns of the assets from a multivariate normal distribution.
Returns a matrix with the samples in rows & each column is an asset.
"""
function generate_data()
    data = zeros(NUM_SAMPLES, NUM_ASSET)
    # Linking factors to induce correlations
    β = [(i-1.0)/NUM_ASSET for i in 1:NUM_ASSET]
    for i in 1:NUM_SAMPLES
        # Common market factor, μ=3%, σ=5%, truncated at ±3σ
        z = rand(Normal(0.03, 0.05))
        z = max(z, 0.03 - 3*0.05)
        z = min(z, 0.03 + 3*0.05)
        for j in 1:NUM_ASSET
            # Idiosyncratic contribution, μ=0%, σ=5%, truncated at ±3σ
            r = rand(Normal(0.00, 0.05))
            r = max(r, 0.00 - 3*0.05)
            r = min(r, 0.00 + 3*0.05)
            data[i,j] = β[j] * z + r
        end
    end
    return data
end


"""
    solve_portfolio(past_returns, Γ)
Solve the robust portfolio problem given a matrix of past returns.
The particular problem solved is
    max   z
    s.t.  ∑ rᵢ xᵢ ≥ z    ∀ r ∈ R      # Worst-case return
          ∑ xᵢ    = 1                 # Is a portfolio
          xᵢ ≥ 0,
where R is one of the uncertainty sets:
    R = { (r,z) | r = μ + Σ^½ z,
            Polyhedral:  ‖z‖₁ ≤ Γ, ‖z‖∞ ≤ 1
           Ellipsoidal:  ‖z‖₂ ≤ Γ
        }
which is controlled by the global flag UNCERTAINY_SET.
"""
function solve_portfolio(past_returns, Γ)
    # Setup the robust optimization model
    m = RobustModel(solver=GurobiSolver())
    # Each asset is a share of the money to invest...
    @variable(m, 0 <= x[1:NUM_ASSET] <= 1)
    # ... and we must allocate all the money.
    @constraint(m, sum(x) == 1)
    # JuMPeR doesn't support uncertain objective functions directly. Instead,
    # an auxiliary variable should be added and the objective function should
    # be placed as a constraint on the auxiliary (epigraph form).
    @variable(m, obj)
    @objective(m, Max, obj)
    # The uncertain parameters are the returns for each asset
    @uncertain(m, r[1:NUM_ASSET])
    # The uncertainty set requires the "square root" of the covariance
    μ = mean(past_returns)  # Want a column vector
    Σ = cov(past_returns)
    L = Σ  # Σ^½
    # Define auxiliary uncertain parameters to model the underlying factors
    @uncertain(m, z[1:NUM_ASSET])
    # Link r with z
    @constraint(m, r .== L*z .+ μ)
    if UNCERTAINY_SET == :Polyhedral
        @constraint(m, norm(z,   1) <= Γ)  # ‖z‖₁ ≤ Γ
        @constraint(m, norm(z, Inf) <= 1)  # ‖z‖∞ ≤ 1
    else
        @constraint(m, norm(z,   2) <= Γ)  # ‖z‖₂ ≤ Γ
    end
    # The objective function: the worst-case return
    @constraint(m, obj <= dot(r, x))
    # Solve it!
    solve(m)
    # Return the allocation and the worst-case return
    return getvalue(x), getvalue(obj)
end

"""
    solve_and_simulate()
Generates past and future returns from the same distribution, builds a
range of portfolios using the past returns, and evaluates them on the future
returns (reporting distribution of returns).
"""
function solve_and_simulate()
    # Generate the simulated data
    past_returns   = generate_data()
    future_returns = generate_data()
    # Generate and store the portfolios
    portfolios = Dict()
    for Γ in 0:NUM_ASSET
        x, _ = solve_portfolio(past_returns, Γ)
        portfolios[Γ] = x
    end
    # Print table header
    println("    Γ |    Min |    10% |    20% |   Mean |    Max | Support")
    # Evaluate and print distributions
    for Γ in 0:NUM_ASSET
        # Evaluate portfolio returns in future
        future_z = future_returns * portfolios[Γ]
        sort!(future_z)
        min_z    = future_z[1]*100
        ten_z    = future_z[div(NUM_SAMPLES,10)]*100
        twenty_z = future_z[div(NUM_SAMPLES, 5)]*100
        mean_z   = mean(future_z)*100
        max_z    = future_z[end]*100
        support  = sum(portfolios[Γ] .> 0.01)  # Number of assets used in portfolio
        @printf(" %4.1f | %6.2f | %6.2f | %6.2f | %6.2f | %6.2f | %7d\n",
                    Γ, min_z, ten_z, twenty_z, mean_z, max_z, support)
    end
end

solve_and_simulate()

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 22 rows, 32 columns and 151 nonzeros
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e-05, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 11 rows and 11 columns
Presolve time: 0.00s
Presolved: 11 rows, 21 columns, 120 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 5.500e+01
 Factor NZ  : 6.600e+01
 Factor Ops : 5.060e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.56410761e-02 -0.00000000e+00  1.00e+00 1.00e-01  5.95e-03     0s
   1   1.28205522e-02  3.47430401e-02  1.10e-06 1.10e-04  7.11e-04

    Γ |    Min |    10% |    20% |   Mean |    Max | Support
  0.0 |  -6.26 |  -2.13 |  -0.82 |   1.42 |   9.78 |      10
  1.0 |  -7.98 |  -2.81 |  -1.81 |   0.42 |   8.35 |       6
  2.0 |  -8.00 |  -2.82 |  -1.81 |   0.42 |   8.34 |       6
  3.0 |  -8.00 |  -2.82 |  -1.82 |   0.42 |   8.34 |       6
  4.0 |  -8.00 |  -2.82 |  -1.82 |   0.42 |   8.33 |       6
  5.0 |  -8.00 |  -2.82 |  -1.82 |   0.42 |   8.33 |       6
  6.0 |  -7.99 |  -2.82 |  -1.81 |   0.42 |   8.34 |       6
  7.0 |  -8.00 |  -2.82 |  -1.81 |   0.42 |   8.34 |       6
  8.0 |  -8.01 |  -2.82 |  -1.82 |   0.42 |   8.34 |       6
  9.0 |  -7.99 |  -2.82 |  -1.81 |   0.42 |   8.34 |       6
 10.0 |  -7.98 |  -2.81 |  -1.82 |   0.42 |   8.33 |       6
